<a href="https://colab.research.google.com/github/014972304505347/iGem2017combinatorialmodel/blob/master/ADC24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'ariel-data-challenge-2024:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F70367%2F9188054%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240803%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240803T235650Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4e77a6799dbb45f00cdaa640ac68167dae92f92f61a312079559ac4992210b1757d8287896ac8bc00fb5aedff96fa88ce6219c4b40d36bbd46928572de07b2e0e4ab8fa1ca70020b6c6843ed55f5e1ba96548b52182db313b3b895118e59c060b514d326dd2bd099a157488d77b3fa0851045a7969b028d4356ae09286afc3f1798b3a037ef5ee0e93be2a4ff989df2080c60206c4826e59557c0ea6d4883c02731a1d4e07be4fe9056eb27f92cbbe74461fb93d54a4d2ccca59ed705218464c106cca29a3ec344c9f54f90bb1dc5e12c7608c6135df6bd0f9026e45b669e35d87587c629ce124c70263cdcba5fd351c36450024864e1fa3747355362a2df0fa'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Ariel Data Challenge 2024: Quickstart ⭐️⭐️⭐️⭐️⭐️

In this notebook, we show how to make predictions using only the FGS1 data.

Of course, we cross-validate our model before preparing a submission.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy.stats
from tqdm import tqdm

from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_squared_error

The following hidden cell contains the function which evaluates the competition metric.

In [ ]:
# Adapted from https://www.kaggle.com/code/metric/ariel-gaussian-log-likelihood
class ParticipantVisibleError(Exception):
    pass

def competition_score(
        solution: pd.DataFrame,
        submission: pd.DataFrame,
        naive_mean: float,
        naive_sigma: float,
        sigma_true: float,
        row_id_column_name='planet_id',
    ) -> float:
    '''
    This is a Gaussian Log Likelihood based metric. For a submission, which contains the predicted mean (x_hat) and variance (x_hat_std),
    we calculate the Gaussian Log-likelihood (GLL) value to the provided ground truth (x). We treat each pair of x_hat,
    x_hat_std as a 1D gaussian, meaning there will be 283 1D gaussian distributions, hence 283 values for each test spectrum,
    the GLL value for one spectrum is the sum of all of them.

    Inputs:
        - solution: Ground Truth spectra (from test set)
            - shape: (nsamples, n_wavelengths)
        - submission: Predicted spectra and errors (from participants)
            - shape: (nsamples, n_wavelengths*2)
        naive_mean: (float) mean from the train set.
        naive_sigma: (float) standard deviation from the train set.
        sigma_true: (float) essentially sets the scale of the outputs.
    '''

    del solution[row_id_column_name]
    del submission[row_id_column_name]

    if submission.min().min() < 0:
        raise ParticipantVisibleError('Negative values in the submission')
    for col in submission.columns:
        if not pd.api.types.is_numeric_dtype(submission[col]):
            raise ParticipantVisibleError(f'Submission column {col} must be a number')

    n_wavelengths = len(solution.columns)
    if len(submission.columns) != n_wavelengths*2:
        raise ParticipantVisibleError('Wrong number of columns in the submission')

    y_pred = submission.iloc[:, :n_wavelengths].values
    # Set a non-zero minimum sigma pred to prevent division by zero errors.
    sigma_pred = np.clip(submission.iloc[:, n_wavelengths:].values, a_min=10**-15, a_max=None)
    y_true = solution.values

    GLL_pred = np.sum(scipy.stats.norm.logpdf(y_true, loc=y_pred, scale=sigma_pred))
    GLL_true = np.sum(scipy.stats.norm.logpdf(y_true, loc=y_true, scale=sigma_true * np.ones_like(y_true)))
    GLL_mean = np.sum(scipy.stats.norm.logpdf(y_true, loc=naive_mean * np.ones_like(y_true), scale=naive_sigma * np.ones_like(y_true)))

    submit_score = (GLL_pred - GLL_mean)/(GLL_true - GLL_mean)
    return float(np.clip(submit_score, 0.0, 1.0))

# Reading the data

We start by reading the metadata:

In [ ]:
train_adc_info = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/train_adc_info.csv',
                           index_col='planet_id')
test_adc_info = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/test_adc_info.csv',
                           index_col='planet_id')
train_labels = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/train_labels.csv',
                           index_col='planet_id')
wavelengths = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/wavelengths.csv')
#axis_info = pd.read_parquet('/kaggle/input/ariel-data-challenge-2024/axis_info.parquet')


Some facts about the data:
- We have 673 planets of 2 stars for training.
- There are roughly 800 planets for testing (but the test data is hidden).
- The competition is a multi-output regression task with 283 wavelengths to predict. The first one is from FGS1, the other 282 are from AIRS.

## Reading and preprocessing the FGS1 data

The FGS1 measurements consist of one file per planet (673 files for 673 planets for training). For now, we ignore the calibration files.

Each file contains 135,000 rows of images taken at 0.1 second time steps. Each row is a 32\*32 image at a single wavelength.

We read a sample file:

In [ ]:
planet_id = 14485303
f_signal = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/train/{planet_id}/FGS1_signal.parquet')
f_signal

Every row of the file corresponds to an image of a star:

In [ ]:
sns.heatmap(f_signal.iloc[1].values.reshape(32, 32))
plt.gca().set_aspect('equal')
plt.show()

To see the time series, we first have to compute the difference between the even and the odd frames to get the net signal. The net signal is very noisy, and we smoothen it by computing a moving average. The plot of the smoothened signal clearly shows that the image gets darker while the planet passes in front of the star (between time steps 22000 and 45000).

In [ ]:
mean_signal = f_signal.values.mean(axis=1)
net_signal = mean_signal[1::2] - mean_signal[0::2]
cum_signal = net_signal.cumsum()
window=800
smooth_signal = (cum_signal[window:] - cum_signal[:-window]) / window

_, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(net_signal, label='raw net signal')
ax1.legend()
ax2.plot(smooth_signal, color='c', label='smoothened net signal')
ax2.legend()
ax2.set_xlabel('time')
plt.suptitle('FGS1 time series', y=0.96)
plt.show()


We now read the FGS1 data for all 673 training planets. We keep only three values for every planet: The mean signal before, during and after the planet passes in front of the star.

In [ ]:
%%time
def f_read_and_preprocess(dataset, adc_info):
    """Read the FGS1 files for all planet_ids and extract the signal.

    Parameters
    dataset: 'train' or 'test'
    adc_info: metadata dataframe, either train_adc_info or test_adc_info

    Returns
    dataframe with one row per planet_id

    """
    planet_ids = adc_info.index
    phases = []
    for i, planet_id in tqdm(list(enumerate(planet_ids))):
        f_signal = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{dataset}/{planet_id}/FGS1_signal.parquet')
        mean_signal = f_signal.values.mean(axis=1) # mean over the 32*32 pixels
        net_signal = mean_signal[1::2] - mean_signal[0::2]

        gain = adc_info.FGS1_adc_gain.values[i]
        phase1 = net_signal[:18000].mean() * gain
        phase2 = net_signal[25000:40000].mean() * gain
        phase3 = net_signal[50000:].mean() * gain

        phases.append((phase1, phase2, phase3))
    return pd.DataFrame(phases, columns=['phase0', 'phase1', 'phase2'])

train = f_read_and_preprocess('train', train_adc_info)


The following scatterplot shows a strong correlation between the signal reduction when the planet is in front of the star and one of the targets we want to predict. The other targets have a similarly high correlation.

In [ ]:
plt.scatter(train.phase0 - train.phase1, train_labels.wl_1, color='g', s=15)
plt.xlabel('signal reduction when planet is in front')
plt.ylabel('target')
plt.title('Correlation between signal reduction and target')
plt.show()

# The model and the cross-validation

To keep things simple, we predict the spectra with ridge regression.

We are interested in three cross-validation metrics:
1. The R2 score is above 0.9, which confirms the correlation we've seen in the scatterplot.
2. The root mean squared error will be the predicted uncertainty.
3. The competition metric gives an indication of the leaderboard score. Unfortunately the competition metric depends on the value of `sigma_true`, which I don't know.

In [ ]:
# model = make_pipeline(PolynomialFeatures(include_bias=False), Ridge())
model = Ridge()

oof_pred = cross_val_predict(model, train, train_labels)

print(f"# R2 score: {r2_score(train_labels, oof_pred):.3f}")
sigma_pred = mean_squared_error(train_labels, oof_pred, squared=False)
print(f"# Root mean squared error: {sigma_pred:.6f}")

col = 1
plt.scatter(oof_pred[:,col], train_labels.iloc[:,col], s=15, c='lightgreen')
plt.gca().set_aspect('equal')
plt.xlabel('y_pred')
plt.ylabel('y_true')
plt.title('Comparing y_true and y_pred')
plt.show()

# R2 score: 0.924
# Root mean squared error: 0.000474

In [ ]:
def make_dataframe(pred_array, index, sigma_pred):
    return pd.concat([pd.DataFrame(pred_array.clip(train_labels.values.min(), train_labels.values.max()), index=index, columns=wavelengths.columns),
                      pd.DataFrame(sigma_pred, index=index, columns=[f"sigma_{i}" for i in range(1, 284)])],
                     axis=1)

oof_df = make_dataframe(oof_pred, train_adc_info.index, sigma_pred)
display(oof_df)

gll_score = competition_score(train_labels.copy().reset_index(),
                              oof_df.copy().reset_index(),
                              naive_mean=train_labels.values.mean(),
                              naive_sigma=train_labels.values.std(),
                              sigma_true=0.0001)
print(f"# Estimated competition score: {gll_score:.3f}")
# Estimated competition score: 0.389

# Submission

In [ ]:
# Refit the model to the full dataset
model.fit(train, train_labels)

# Predict
test = f_read_and_preprocess('test', test_adc_info)
test_pred = model.predict(test)

# Package into submission file
sub = make_dataframe(test_pred, test_adc_info.index, sigma_pred)
display(sub)
sub.to_csv('submission.csv')
#!head submission.csv

# Outlook: The AIRS measurements

AIRS is the other sensor of the satellite. Again, it produces one file per planet.

Each file contains 11,250 rows of images captured at constant time steps. Each 32 x 356 image has been flattened into 11392 columns.

In [ ]:
a_signal = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/train/{planet_id}/AIRS-CH0_signal.parquet')
a_signal

In [ ]:
a_signal = a_signal.values.reshape(11250, 32, 356)

In [ ]:
plt.figure(figsize=(10, 3))
sns.heatmap(a_signal[1])
plt.ylabel('spatial dimension')
plt.xlabel('wavelength dimension')
plt.show()

The data again is a time series, and we can see the star being obscured while the planet is passing in front of it.

In [ ]:
mean_signal = a_signal.mean(axis=2).mean(axis=1)
net_signal = mean_signal[1::2] - mean_signal[0::2]
cum_signal = net_signal.cumsum()
window=80
smooth_signal = (cum_signal[window:] - cum_signal[:-window]) / window

_, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(net_signal, label='raw net signal')
ax1.legend()
ax2.plot(smooth_signal, color='c', label='smoothened net signal')
ax2.legend()
ax2.set_xlabel('time')
plt.suptitle('AIRS-CH0 time series', y=0.96)
plt.show()
